In [13]:
import time
import math
from datetime import datetime

In [2]:
def number_range(x,rmin,rmax):
    
    shift_x = x-rmin
    delta = rmax-rmin
    
#     print(shift_x,delta)
    
    return (((shift_x % delta) +delta) % delta) + rmin

In [3]:
number_range(500, 10,1000)

500

In [4]:
490 % 990, 490 % 990 + 990, (((490 % 990) +990) % 990)

(490, 1480, 490)

In [5]:
def sun_position_dnum(dnum, lat, lon, debug=False):
    
    #constants
    tau = math.pi * 2.0
    rpd = math.pi / 180.0
    
    # lat / lon in radians
    rlat = lat * rpd
    rlon = lon * rpd
    if debug: print(f'rlat: {rlat}, rlon:{rlon}')

    if debug: print(f'dnum: {dnum}')
    
    #mean lon of sun
    slon =  number_range(dnum * 0.01720279239 + 4.894967873,0, tau)
    if debug: print(f'slon: {slon}')
    
    # mean anomaly of sun
    sano = number_range(dnum * 0.01720197034 + 6.240040768 ,0, tau)
    if debug: print(f'sano: {sano}')
    
    # ecliptic lon of the sun
    ecli = slon + 0.03342305518 * math.sin(sano) + 0.0003490658504 * math.sin(2*sano)
    if debug : print(f'ecli: {ecli}')
        
    #obliquity of the ecliptic
    obli = 0.4090877234 - 0.000000006981317008 * dnum
    if debug: print(f'obli: {obli}')
    
    #right ascension of the sun
    tmpx = math.cos(obli) * math.sin(ecli)
    tmpy = math.cos(ecli)
    rasc = math.atan2(tmpx,tmpy)
    
    if debug: print(f'rasc: {rasc}')
        
    #declination
    decl = math.asin(math.sin(obli) * math.sin(ecli))
    if debug: print(f'decl: {decl}')
    
    #local sidereal time
    stim = number_range(4.894961213 + 6.300388099 * dnum + rlon ,0,tau)
    if debug: print(f'stim: {stim}')
    
    #hour angle of sun
    hang = number_range(stim - rasc,0,tau)
    if debug: print(f'hang: {hang}')
    
    # local elevation of sun
    tmpx = math.cos(decl) *math.cos(rlat) *math.cos(hang)
    elevation = math.asin(math.sin(decl) * math.sin(rlat) +tmpx )
    if debug: print(f'elevation: {elevation}')
    
    #local azimuth of the sun
    tmpx = -math.cos(decl) * math.cos(rlat) * math.sin(hang)
    tmpy =  math.sin(decl) -math.sin(rlat) *math.sin(elevation)
    azimuth = math.atan2(tmpx,tmpy)
    if debug: print(f'azimuth: {azimuth}')
    
    # azimuth & elevation in degrees
    azi_deg = number_range(azimuth / rpd ,0.0, 360.0)
    ele_deg = number_range(elevation / rpd, 0.0, 360.0)
    
    if debug: print(f'ele_deg: {ele_deg}')
    
    #refraction correction
    targ = (ele_deg +(10.3 / (ele_deg +5.11 ))) * rpd
    if debug: print(f'targ: {targ}')
        
    refraction = (1.02 / math.tan(targ)) / 60.0
    
    #adjust elevation for refraction
    ele_deg_refract = ele_deg + refraction
    
    return azi_deg, ele_deg_refract, refraction

In [51]:
book_ticks = 1576084148.0

In [11]:
tick_site = 1576058948

In [12]:
ticks = time.mktime((2019,12,11,10,9,8,0,0,0))
ticks

1576058948.0

In [13]:
(book_ticks - ticks) / 60 / 60

7.0

In [14]:
(tick_site - ticks)/60/60

0.0

In [44]:
# ticks = time.mktime((2019,12,11,10,9,8,0,0,0))



In [20]:
def dnum_route(year, month, day, hour, mins, sec, tz):
    
    dy1 = math.floor((month + 9) /12)
    dy2 = math.floor(7* (year + dy1 )/4)
    dy3 = math.floor(275 * month /9)
    dy4 = 367 * year - dy2 + dy3
    
    utm = hour - tz +mins/60 + sec / 3600
    
    return dy4 +day  - 730531.5 + utm /24

def dnum_dt(dt,tz):
    
    dy1 = math.floor((dt.month + 9) /12)
    dy2 = math.floor(7* (dt.year + dy1 )/4)
    dy3 = math.floor(275 * dt.month /9)
    dy4 = 367 * dt.year - dy2 + dy3
    
    utm = dt.hour - tz +dt.minute/60 + dt.second / 3600
    
    return dy4 +dt.day  - 730531.5 + utm /24

In [21]:
dt_test = datetime(2019,12,11,10,9,8)
dnum_dt(dt_test,-7)

7284.214675925926

In [7]:
dnum = dnum_route(2019,12,11,10,9,8,-7)
tz_num=-7.0
lat = 40.6027778
lon = -104.7416667
dnum

7284.214675925926

In [8]:
sun_position_dnum(dnum, lat, lon, debug=True)

rlat: 0.7086521580656596, rlon:-1.8280869479415036
dnum: 7284.214675925926
slon: 4.540094523553108
sano: 5.879179429878775
ecli: 4.5267034130623935
obli: 0.40903686998819305
rasc: -1.7727450582417488
decl: -0.40159712492436245
stim: 4.060845390204861
hang: 5.833590448446611
elevation: 0.3843864905104998
azimuth: 2.6954257399905615
ele_deg: 22.0237236080971
targ: 0.39101178512733475


(154.43651889238595, 22.064961906859356, 0.0412382987622563)

In [9]:
dnum = dnum_route(2023,5,28,13,0,0,1)
lat = 50
lon = 0

In [10]:
sun_position_dnum(dnum, lat, lon, debug=True)

rlat: 0.8726646259971648, rlon:0.0
dnum: 8548.0
slon: 1.1479898504099424
sano: 2.486035862009942
ecli: 1.168027206428651
obli: 0.40902804710221563
rasc: 1.1360585150543936
decl: 0.37459110335950363
stim: 1.1479783215873738
hang: 0.011919806532979749
elevation: 1.072633855603225
azimuth: -3.1183743464869362
ele_deg: 61.457392888909794
targ: 1.0753344107121723


(181.33031100443236, 61.466580106482155, 0.009187217572362988)

In [66]:
# nelson walk
lat =51.52371
lon = -0.01381

for hr in range(6,18,1):
    dnum = dnum_route(2023,5,30,hr,0,0,1)
    azi, ele, _ = sun_position_dnum(dnum,lat,lon)
    
    print(ele,azi)

8.512691492500567 65.4495876995191
17.27508276386644 76.52147905556853
26.488855039885504 87.77369597495903
35.77800765689599 99.91432406852698
44.691044474392704 113.97790240532908
52.54404371613678 131.4679793644978
58.22049696744939 154.00635358959812
60.253786021843595 181.12914841878103
57.891425226828545 208.0180912776857
51.996050724196714 230.12948139359708
44.031266870368924 247.28955602195913
35.07489316070655 261.15747963995636


### tests

In [11]:
tests = [
    {'source':'book example','lat':40.6027778,'lon':-104.7416667,
     'year':2019,'month':12,'day':11,'hour':10,'min':9,'sec':8,'tz':-7,
    'expected_elevation':22.064 ,'expected_azimuth':154.436},
        {'source':'creation day','lat':50,'lon':0,
     'year':2023,'month':5,'day':28,'hour':13,'min':0,'sec':0,'tz':1,
    'expected_elevation':61.47 ,'expected_azimuth':181.33}
]

In [12]:
for test in tests:
    print(test['source'])
    dnum = dnum_route(test['year'],test['month'],test['day'],test['hour'],test['min'],test['sec'],test['tz'])
    azi, ele, _ = sun_position_dnum(dnum, test['lat'],test['lon'])
  
    abs_err_azi = abs(azi-test['expected_azimuth'])
    abs_err_ele  =abs(ele-test['expected_elevation'])
    
    print(f'Azi: {abs_err_azi}, Ele: {abs_err_ele}')
    assert  abs_err_azi<1
    assert  abs_err_ele<1
    

book example
Azi: 0.0005188923859407168, Ele: 0.0009619068593558211
creation day
Azi: 0.0003110044323477723, Ele: 0.003419893517843775


In [13]:
from datetime import datetime
import pytz # $ pip install pytz



In [14]:
naive_dt = datetime.strptime('2019-12-11 10:09:08', '%Y-%m-%d %H:%M:%S')
tz = pytz.timezone('US/Mountain')
# tz = pytz.timezone('UTC')

mountain_dt = tz.normalize(tz.localize(naive_dt))
print(mountain_dt)

timestamp = (mountain_dt - datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds()
timestamp, (timestamp-book_ticks)/60/60

2019-12-11 10:09:08-07:00


(1576084148.0, 0.0)

In [22]:
from sun_position import test_sun_position

In [24]:
test_sun_position()